In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

In [2]:
"定义卷积"
def corr2d(X, K):
    Y = torch.zeros((X.shape[0] - K.shape[0] + 1, X.shape[1] - K.shape[1] + 1), dtype = torch.float32)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + K.shape[0], j:j + K.shape[1]] * K).sum()
    return Y

### 1. 多输入单输出卷积

In [3]:
"多输入卷积"
def corr2d_multi_in(X, K):
    y = torch.zeros((X.shape[1] - K.shape[1] + 1, X.shape[2] - K.shape[2] + 1), dtype = torch.float32)
    for i, k in zip(X, K):
        y[:] = corr2d(i, k) + y
    return y

In [4]:
x = torch.tensor([[[0, 1 ,2], [3, 4, 5], [6, 7, 8]], [[1, 2 ,3], [4, 5, 6], [7, 8, 9]]], dtype = torch.float32)
k = torch.tensor([[[0, 1], [2, 3]], [[1, 2], [3, 4]]], dtype = torch.float32)
x.shape, k.shape, x, k

(torch.Size([2, 3, 3]),
 torch.Size([2, 2, 2]),
 tensor([[[0., 1., 2.],
          [3., 4., 5.],
          [6., 7., 8.]],
 
         [[1., 2., 3.],
          [4., 5., 6.],
          [7., 8., 9.]]]),
 tensor([[[0., 1.],
          [2., 3.]],
 
         [[1., 2.],
          [3., 4.]]]))

In [5]:
corr2d_multi_in(x, k)

tensor([[ 56.,  72.],
        [104., 120.]])

### 2. 多输入多输出卷积

In [6]:
x = torch.tensor([[[0, 1 ,2], [3, 4, 5], [6, 7, 8]], [[1, 2 ,3], [4, 5, 6], [7, 8, 9]]], dtype = torch.float32)
k = torch.stack((k, k + 1, k + 2), 0)
x.shape, k.shape, x, k

(torch.Size([2, 3, 3]),
 torch.Size([3, 2, 2, 2]),
 tensor([[[0., 1., 2.],
          [3., 4., 5.],
          [6., 7., 8.]],
 
         [[1., 2., 3.],
          [4., 5., 6.],
          [7., 8., 9.]]]),
 tensor([[[[0., 1.],
           [2., 3.]],
 
          [[1., 2.],
           [3., 4.]]],
 
 
         [[[1., 2.],
           [3., 4.]],
 
          [[2., 3.],
           [4., 5.]]],
 
 
         [[[2., 3.],
           [4., 5.]],
 
          [[3., 4.],
           [5., 6.]]]]))

In [9]:
def corr2d_multi_in_out(X, K):
    y = []
    for i in range(len(K)):
        a = corr2d_multi_in(X, K[i])
        y.append(a)
    return torch.cat(y,dim=0).reshape(-1, X.shape[1] - K.shape[2] + 1, X.shape[2] - K.shape[3] + 1)

In [10]:
corr2d_multi_in_out(x, k)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

### 3. 1X1卷积层

In [11]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape(-1, h * w)
    K = K.reshape(c_o, c_i)
    y = torch.mm(K, X)
    return y.reshape(c_o, h, w)

In [12]:
x = torch.normal(0 ,1, (3, 3, 3))
k = torch.normal(0, 1, (2, 3, 1, 1))

In [14]:
corr2d_multi_in_out_1x1(x, k) == corr2d_multi_in_out(x, k)

tensor([[[True, True, True],
         [True, True, True],
         [True, True, True]],

        [[True, True, True],
         [True, True, True],
         [True, True, True]]])